In [1]:
!pip install tensorflow

     -------------------------------------- 300.8/300.8 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 44.2 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 26.3 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 31.4 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 53.1 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 130.2/130.2 kB ? eta 0:00:00
     ------------------------------------- 938.4/938.4 kB 29.9 MB/s eta 0:00:00
     ------------------------------------- 442.0/442.0 kB 27.0 MB/s eta 0:00:00
     --------------------------------------- 24.4/24.4 MB 31.2 MB/s eta 0:00:00
     ------------------------------------- 413.4/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.2 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.2 which is incompatible.


In [2]:
pip install opencv--python

     --------------------------------------- 38.1/38.1 MB 25.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os
import matplotlib as plt
import cv2 as cv
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn import metrics

D:\Anaconda2\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
%cd "D://250x250//"

D:\250x250


In [4]:
path = "D://Downloads//250x250"
# Load file names and labels for the processed data
data_folder_prefix = "Seg_Featured_Data/Seg_Featured_"

with open("data_labels.json", 'r') as json_file:
    label_dict = json.load(json_file)


file_names = list(label_dict.keys())
print(len(file_names))

labels = []
for fl in file_names:
    labels.append(label_dict[fl])
print(len(labels))
test_load = np.load("D://Downloads//test0.npy", None, allow_pickle=True)
test_load.shape

files = ["Seg_Featured_"+x+".npy" for x in file_names]
# files = glob.glob("Seg_Featured_Data/*.npy")
# files = glob.glob("Seg_Featured_Data/*.npy")
max_len = 250*250+7
num_padded = 0
flattened_test_load = np.zeros((max_len, ))
print(flattened_test_load.shape)

for fl in files:
    test_load = np.load(fl, None, allow_pickle=True)
    y,x,_ = test_load.shape
    # Ignore depth 0
    # Keep all Depth 1
    #  if the x,y is less than 250x50 pad w 0s
    layers_depth = test_load[:,:,1]
    # print(layers_depth.shape)
    if x < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((y, y-x))))
        num_padded +=1
    if y < 250:
        layers_depth = np.hstack((layers_depth, np.zeros((250-y, 250))))
    layers_depth = layers_depth.flatten()
    # Get the metrics from the other depths
    vl, ct = np.unique(test_load[:,:,5], return_counts=True)
    if len(ct)>1:
        num_bp = ct[1]
    else:
        num_bp = 0
    # print("conected components (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]))
    # print("orientaiton (mean, std)", np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]))
    # print("mean orientaiton (mean, std)", np.mean(test_load[:250//10-1,:250//10-1,4]), np.std(test_load[:250//10-1,:250//10-1,4]))
    # print("num breakpoints",num_bp)
    # print("dist map (mean, std)", np.mean(test_load[:,:,6]), np.std(test_load[:,:,6]))
    data= np.append(layers_depth, [np.mean(test_load[:y//10-1,:x//10-1,2]), np.std(test_load[:y//10-1,:x//10-1,2]),
                                    np.mean(test_load[:y//10-1,:x//10-1,3]), np.std(test_load[:y//10-1,:x//10-1,3]),
                                    num_bp,
                                    np.mean(test_load[:,:,6]), np.std(test_load[:,:,6])])

    # print(data.shape)
    flattened_test_load = np.vstack((flattened_test_load, data))
print(flattened_test_load.shape,flattened_test_load[0].shape, "num padded:", num_padded)
flattened_data = flattened_test_load[1:,:]
flattened_data.shape, len(labels)

743
743
(62507,)
(744, 62507) (62507,) num padded: 106


((743, 62507), 743)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(flattened_data, labels, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
y_test.shape

(149,)

In [6]:
model = Sequential()
model.add(Dense(64, input_dim=62507, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [8]:
model2 = Sequential()
model2.add(Dense(128,input_dim=62507,activation='relu'))
model2.add(Dense(64,activation='relu'))
model2.add(Dense(32,activation='relu'))
model2.add(Dense(1,activation='linear'))

In [9]:
model3 = Sequential()
model3.add(Dense(256,input_dim=62507,activation='relu'))
model3.add(Dense(128,activation='relu'))
model3.add(Dense(64,activation='relu'))
model3.add(Dense(32,activation='relu'))
model3.add(Dense(1,activation='linear'))

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

In [11]:
model2.compile(loss='mean_squared_error',optimizer='adam')

In [12]:
model3.compile(loss='mean_squared_error',optimizer='adam')

In [13]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


19/19 [==============================] - 2s 37ms/step - loss: 3.2483 - accuracy: 0.0421
Epoch 2/10
19/19 [==============================] - 1s 36ms/step - loss: 3.0240 - accuracy: 0.0539
Epoch 3/10
19/19 [==============================] - 1s 36ms/step - loss: 1.6079 - accuracy: 0.0657
Epoch 4/10
19/19 [==============================] - 1s 36ms/step - loss: 0.5181 - accuracy: 0.0623
Epoch 5/10
19/19 [==============================] - 1s 35ms/step - loss: 0.2993 - accuracy: 0.0673
Epoch 6/10
19/19 [==============================] - 1s 35ms/step - loss: 0.2503 - accuracy: 0.0673
Epoch 7/10
19/19 [==============================] - 1s 35ms/step - loss: 0.1667 - accuracy: 0.0673
Epoch 8/10
19/19 [==============================] - 1s 34ms/step - loss: 0.2024 - accuracy: 0.0673
Epoch 9/10
19/19 [==============================] - 1s 35ms/step - loss: 0.3687 - accuracy: 0.0673
Epoch 10/10
19/19 [==============================] - 1s 36ms/step - loss: 0.4943 - accuracy: 0.0673


In [14]:
model2.fit(X_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
19/19 [==============================] - 2s 69ms/step - loss: 4.0346
Epoch 2/10
19/19 [==============================] - 1s 68ms/step - loss: 2.7470
Epoch 3/10
19/19 [==============================] - 1s 69ms/step - loss: 1.1341
Epoch 4/10
19/19 [==============================] - 1s 69ms/step - loss: 0.7811
Epoch 5/10
19/19 [==============================] - 1s 69ms/step - loss: 0.5161
Epoch 6/10
19/19 [==============================] - 1s 68ms/step - loss: 0.2346
Epoch 7/10
19/19 [==============================] - 1s 68ms/step - loss: 0.0991
Epoch 8/10
19/19 [==============================] - 1s 68ms/step - loss: 0.0441
Epoch 9/10
19/19 [==============================] - 1s 67ms/step - loss: 0.0347
Epoch 10/10
19/19 [==============================] - 1s 67ms/step - loss: 0.0266


In [15]:
model3.fit(X_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
10/10 [==============================] - 3s 139ms/step - loss: 5.8408
Epoch 2/10
10/10 [==============================] - 1s 138ms/step - loss: 1.9100
Epoch 3/10
10/10 [==============================] - 1s 137ms/step - loss: 0.7247
Epoch 4/10
10/10 [==============================] - 1s 137ms/step - loss: 0.4650
Epoch 5/10
10/10 [==============================] - 1s 138ms/step - loss: 0.3204
Epoch 6/10
10/10 [==============================] - 1s 140ms/step - loss: 0.1714
Epoch 7/10
10/10 [==============================] - 1s 139ms/step - loss: 0.0898
Epoch 8/10
10/10 [==============================] - 1s 137ms/step - loss: 0.0526
Epoch 9/10
10/10 [==============================] - 1s 139ms/step - loss: 0.0294
Epoch 10/10
10/10 [==============================] - 1s 136ms/step - loss: 0.0187


In [16]:
pred = model.predict(X_test)
pred.shape

5/5 [==============================] - 0s 5ms/step


(149, 1)

In [17]:
pred2 = model2.predict(X_test)
pred2.shape

5/5 [==============================] - 0s 7ms/step


(149, 1)

In [26]:
pred3 = model3.predict(X_test)
pred3.shape

5/5 [==============================] - 0s 11ms/step


(149, 1)

In [29]:
print(metrics.precision_score(y_test,np.rint(pred),average="micro"))
print(metrics.precision_score(np.rint(pred),y_test,average="macro"))
print(metrics.recall_score(np.rint(pred),y_test,average="micro"))
print(metrics.recall_score(np.rint(pred),y_test,average="macro"))
print(metrics.accuracy_score(np.rint(pred),y_test))
print(metrics.mean_squared_error(y_test,pred))
print(metrics.mean_absolute_error(y_test,pred))
print(metrics.f1_score(y_test,np.rint(pred),average="micro"))
print(metrics.f1_score(y_test,np.rint(pred),average="macro"))

0.2080536912751678
0.17159361196058442
0.2080536912751678
0.19297746021883955
0.2080536912751678
2.6203861268068978
1.3188219953503384
0.2080536912751678
0.13650600900145526


D:\Anaconda2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
print(metrics.precision_score(y_test,np.rint(pred2),average="micro"))
print(metrics.precision_score(np.rint(pred2),y_test,average="macro"))
print(metrics.recall_score(np.rint(pred2),y_test,average="micro"))
print(metrics.accuracy_score(np.rint(pred2),y_test))
print(metrics.recall_score(np.rint(pred),y_test,average="macro"))
print(metrics.mean_squared_error(y_test,pred2))
print(metrics.mean_absolute_error(y_test,pred2))
print(metrics.f1_score(y_test,np.rint(pred2),average="micro"))
print(metrics.f1_score(y_test,np.rint(pred2),average="macro"))

0.2348993288590604
0.27157055482743553
0.2348993288590604
0.2348993288590604
0.19297746021883955
2.23684118153881
1.276738361224232
0.2348993288590604
0.196790403009979


D:\Anaconda2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print(metrics.precision_score(y_test,np.rint(pred3),average="micro"))
print(metrics.precision_score(np.rint(pred3),y_test,average="macro"))
print(metrics.recall_score(np.rint(pred3),y_test,average="micro"))
print(metrics.recall_score(np.rint(pred3),y_test,average="macro"))
print(metrics.accuracy_score(np.rint(pred3),y_test))
print(metrics.mean_squared_error(y_test,pred3))
print(metrics.mean_absolute_error(y_test,pred3))
print(metrics.f1_score(y_test,np.rint(pred3),average="micro"))
print(metrics.f1_score(y_test,np.rint(pred3),average="macro"))

0.24161073825503357
0.24574049803407602
0.24161073825503357
0.2596070148489503
0.24161073825503357
2.069403368817958
1.197907934652879
0.24161073825503357
0.1948170935040683


D:\Anaconda2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
